#### 1. 라이브러리 불러오기

In [9]:
import torch
from datasets import load_dataset
from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments

import time
import psutil

In [10]:
print(torch.cuda.is_available())

True


#### 2. 기본 모델 불러오기

In [11]:
model_name = "bert-base-uncased"
model_original = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### 3. LoRA 설정

In [12]:
# 자세한 매개변수 설명은 아래 링크 참조
# https://huggingface.co/docs/peft/v0.14.0/en/package_reference/lora#peft.LoraConfig 
# https://huggingface.co/docs/peft/package_reference/peft_types

lora_config = LoraConfig(
    r=4,
    # Transformer architecture에서 query, key, value, output 행렬 중 어떤 행렬에 대해서 LoRA를 적용할 것인지 선택
    target_modules=["query", "value"],
    task_type=TaskType.SEQ_CLS,
    lora_alpha=32,
    lora_dropout=0.05
)

model_lora = get_peft_model(model=model_original, peft_config=lora_config)

model_lora.print_trainable_parameters()

trainable params: 148,994 || all params: 109,632,772 || trainable%: 0.1359


#### 4. 데이터셋 불러오기

In [13]:
# 텍스트 분류 (Text Classification)
# IMDb: 영화 리뷰 감성 분석 (긍정 / 부정)
dataset = load_dataset("imdb")


# 데이터 전처리 함수
def preprocess_function(tokenizer, data):
    return tokenizer(data["text"], padding="max_length", truncation=True)

# 100개만 사용
train_dataset = dataset["train"].select(range(100)).map(
    lambda data: preprocess_function(tokenizer, data), batched=True)

eval_dataset = dataset["test"].select(range(100)).map(
    lambda data: preprocess_function(tokenizer, data), batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

#### 5. 학습

In [14]:
def get_memory_usage():
    process = psutil.Process()
    memory_info = process.memory_info()
    
    return memory_info.rss / (1024 ** 2)

In [15]:
# 1. 기본 모델 bert-base-uncased
start_time_base = time.time()
start_memory_base = get_memory_usage()

training_args_base = TrainingArguments(
    output_dir="./results/base",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs/base",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    seed=2024,
)

trainer_base = Trainer(
    model=model_original,
    args=training_args_base,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

trainer_base.train()

end_time_base = time.time()
time_base = end_time_base - start_time_base

end_memory_base = get_memory_usage()
memory_base = end_memory_base - start_memory_base

print(f"Base Model Training Time: {time_base} seconds")
print(f"Base Model Memory Usage: {memory_base} MB")

C:\Users\MSI\AppData\Local\Temp\ipykernel_16604\1202334515.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_base = Trainer(


  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.03844061121344566, 'eval_runtime': 11.9034, 'eval_samples_per_second': 8.401, 'eval_steps_per_second': 0.336, 'epoch': 1.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.008409840054810047, 'eval_runtime': 10.2117, 'eval_samples_per_second': 9.793, 'eval_steps_per_second': 0.392, 'epoch': 2.0}
{'train_runtime': 140.3825, 'train_samples_per_second': 1.425, 'train_steps_per_second': 0.057, 'train_loss': 0.12301313132047653, 'epoch': 2.0}
Base Model Training Time: 141.12345552444458 seconds
Base Model Memory Usage: 846.1328125 MB


In [16]:
# 2. LoRA 적용 모델
start_time_lora = time.time()
start_memory_lora = get_memory_usage()

training_args_lora = TrainingArguments(
    output_dir="./results/LoRA",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs/LoRA",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    seed=2024,
)

trainer_lora = Trainer(
    model=model_lora,
    args=training_args_lora,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

trainer_lora.train()

end_time_lora = time.time()
time_lora = end_time_lora - start_time_lora

end_memory_lora = get_memory_usage()
memory_lora = end_memory_lora - start_memory_lora

print(f"LoRA Model Training Time: {time_lora} seconds")
print(f"LoRA Model Memory Usage: {memory_lora} MB")

C:\Users\MSI\AppData\Local\Temp\ipykernel_16604\3085779995.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_lora = Trainer(


  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.00016954993770923465, 'eval_runtime': 80.9768, 'eval_samples_per_second': 1.235, 'eval_steps_per_second': 0.049, 'epoch': 1.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 7.135372288757935e-05, 'eval_runtime': 58.8425, 'eval_samples_per_second': 1.699, 'eval_steps_per_second': 0.068, 'epoch': 2.0}
{'train_runtime': 258.0062, 'train_samples_per_second': 0.775, 'train_steps_per_second': 0.031, 'train_loss': 0.001580627984367311, 'epoch': 2.0}
LoRA Model Training Time: 258.6091229915619 seconds
LoRA Model Memory Usage: -64.5390625 MB


#### 6. 성능 비교하기

In [17]:
results_base = trainer_base.evaluate()
results_lora = trainer_lora.evaluate()

print("기본 모델 성능:")
print(results_base)

print("LoRA 모델 성능:")
print(results_lora)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

기본 모델 성능:
{'eval_loss': 7.135372288757935e-05, 'eval_runtime': 58.2201, 'eval_samples_per_second': 1.718, 'eval_steps_per_second': 0.069, 'epoch': 2.0}
LoRA 모델 성능:
{'eval_loss': 7.135372288757935e-05, 'eval_runtime': 66.7357, 'eval_samples_per_second': 1.498, 'eval_steps_per_second': 0.06, 'epoch': 2.0}
